### **유저 세그멘테이션(User Segmentation) 및 프로파일링(profiling) 분석**

데이터베이스 분석에서 유저 세그멘테이션과 프로파일링은 사용자를 더 깊이 이해하고, 이를 바탕으로 맞춤화된 전략을 수립하는 데 필수적인 기법입니다.


**유저 세그멘테이션 (User Segmentation)**

유저 세그멘테이션은 공통된 특성을 가진 사용자들을 그룹화하여 분석하는 방식입니다. 이를 통해 전체 사용자를 더 작은 단위로 나누어 각 그룹의 특징을 발견하고 마케팅에 활용할 수 있습니다.

**유저 프로파일링 (User Profiling)**

유저 프로파일링은 사용자의 행동, 선호도, 특성 등을 기반으로 상세하고 데이터 중심적인 설명 또는 프로필을 생성하는 과정입니다. 이를 통해 사용자의 니즈, 관심사, 동기 등을 깊이 이해할 수 있습니다.


1. 거래 행동 기반 사용자 군집화(KMeans 등)
2. 각 세그먼트별 거래 특성 및 리스크 프로파일 정의
3. 정상 사용자 vs 고위험 사용자 행동 차이 비교

---